# Classify Wine Quality using FLAML

# FLAML을 이용한 와인 등급 분류

## Introduction

인공지능은 프로세스를 자동화하고, 비즈니스에 대한 통찰력을 모으고, 프로세스 속도를 높이기 위해 다양한 산업에서 사용되고 있습니다. 인공지능이 실제로 산업에 어떤 영향을 미치는지 실제 시나리오에서 인공지능의 사용을 연구하기 위해 Python을 사용할 것입니다.

와인 등급 분류는 와인 산업에서는 매우 중요한 문제입니다. 와인 양조장에서는 다양한 데이터를 지속적으로 모니터링하면서 그 등급을 관리하고 있습니다. 비록 우리가 와인 생산자는 아니지만, 만일 우리에게 와인 자료가 주어진다면 그 와인이 상급인지(구입할 것인지) 아니면 하급인지(구입하지 않을 것인지)를 판단하는데 도움을 줄 수 있을 것입니다. 실제로 이런 방법으로 와인을 구매하지는 않지만, 머신러닝의 지도학습 중에서 분류 방법을 공부하는데 도움이 될 수 있습니다. 


이 노트북에서는 FLAML을 사용한 다양한 머신러닝 알고리즘을 사용한 와인 등급 분류에 관하여 학습합니다.


## Context

우리는 [Kaggle](https://www.kaggle.com/datasets/uciml/red-wine-quality-cortez-et-al-2009)에서 얻은 레드와인 데이터로 실습할 것입니다. Kaggle은 데이터 전문가들이 모여 지식을 공유하고 서로 경쟁하여 보상을 받을 수 있는 데이터 공유 플랫폼입니다. 학습의 편의를 위해 위 사이트에서 다운로드 받은 데이터가 '[Dataset]\_Module8\_(redWine).csv'로 제공됩니다.

## 필요한 패키지 설치

In [2]:
#FLAML 설치
!pip install flaml[notebook] --quiet

     |████████████████████████████████| 199 kB 7.4 MB/s 
     |████████████████████████████████| 2.0 MB 52.4 MB/s 
     |████████████████████████████████| 158 kB 44.9 MB/s 
     |████████████████████████████████| 76.6 MB 1.2 MB/s 
     |████████████████████████████████| 757 kB 76.4 MB/s 


## Import Libraries

In [1]:
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import flaml

## Use Python to open csv files

Pandas는 데이터 과학을 위한 인기 있는 Python 라이브러리입니다. 강력하고 유연한 데이터 구조를 제공하여 데이터 조작 및 분석을 더 쉽게 만듭니다.

## Read the CSV file

In [6]:
wine_data = pd.read_csv(r'[Dataset]_Module8_(redWine).csv')
wine_data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


#데이터 개수 
-> df.info() -> 1599개

12개의 columns 
-> 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      
 2   citric acid           
 3   residual sugar        
 4   chlorides             
 5   free sulfur dioxide   
 6   total sulfur dioxide  
 7   density               
 8   pH                    
 9   sulphates             
 10  alcohol               
 11  quality               

In [8]:
wine_data.info()

array([5, 6, 7, 4, 8, 3], dtype=int64)

# 등급별 개수 조사

In [29]:
n_equal = wine_data['quality'].value_counts() # number of each quality class
print(n_equal)

5    681
6    638
7    199
4     53
8     18
3     10
Name: quality, dtype: int64


# * 3~8 등급의 와인 품질을 상급과 하급의 2진 분류 문제로 변환

## 데이터 준비와 분할

In [30]:
# 와인 품질 5 초과 상급, 5이하 하급으로 변경 (2진 분류)
# quality 6,7,8 -> grade 1.0 , quality 3,4,5 -> grade 0.0 
y=pd.Series([1.0 if v_equal>5 else 0.0 for v_equal in wine_data['quality']])
# label로 사용

In [31]:
# feature data
X = wine_data.drop(['quality'], axis=1)

In [32]:
# 데이터를 학습용과 시험용으로 분할
X_train, X_test, y_train, y_test = train_test_split(
    StandardScaler().fit_transform(X), y, random_state=1)

## 4. 분류기 모델 생성

In [38]:
# AutoML 도구인 FLAML을 사용하여 최적 모델 자동 판별
aml1 = flaml.AutoML()
aml1.fit(X_train, y_train, task="classification")

[flaml.automl: 08-08 09:16:26] {2444} INFO - task = classification
INFO:flaml.automl:task = classification
[flaml.automl: 08-08 09:16:26] {2446} INFO - Data split method: stratified
INFO:flaml.automl:Data split method: stratified
[flaml.automl: 08-08 09:16:26] {2449} INFO - Evaluation method: holdout
INFO:flaml.automl:Evaluation method: holdout
[flaml.automl: 08-08 09:16:26] {2568} INFO - Minimizing error metric: 1-roc_auc
INFO:flaml.automl:Minimizing error metric: 1-roc_auc
[flaml.automl: 08-08 09:16:26] {2617} WARNING - No search budget is provided via time_budget or max_iter. Training only one model per estimator. To tune hyperparameters for each estimator, please provide budget either via time_budget or max_iter.
INFO:flaml.default.suggest:metafeature distance: 0.05527505800319427
INFO:flaml.default.suggest:metafeature distance: 0.05527505800319427
INFO:flaml.default.suggest:metafeature distance: 0.05527505800319427
INFO:flaml.default.suggest:metafeature distance: 0.055275058003194

## 5. 평가

머신러닝 알고리즘의 정확도는 알고리즘이 얼마나 잘 수행되고 있는지, 즉 알고리즘이 데이터 포인트를 올바르게 분류하는 빈도를 측정하는 것입니다. 정확도는 다음과 같이 주어집니다:
![정확도](https://miro.medium.com/max/1050/1*O5eXoV-SePhZ30AbCikXHw.png)
정밀도는 관련성 있는 결과의 %를 의미하고, 재현율은 알고리즘에 의해 올바르게 분류된 전체 관련 결과의 %를 의미합니다.
![Precision and Recall](https://miro.medium.com/max/1050/1*pOtBHai4jFd-ujaNXPilRg.png)
True positive: 모델이 긍정 클래스를 올바르게 예측합니다.<br>
True negative: 모델이 부정 클래스를 올바르게 예측합니다.<br>
False positiv: 모델이 긍정 클래스를 잘못 예측합니다.<br>
False negative: 모델이 부정 클래스를 잘못 예측합니다

In [39]:
#train data에 의해 생성된 모델을 이용하여 test data로 예측
y_pred = aml1.predict(X_test)
print(sum(abs(y_test - y_pred)))  #잘못 예측한 데이터 갯수 의미

81.0


In [40]:
#예측된 값과 실제값 사이의 오차 성능평가 보고서 (Accuracy 등)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.78      0.79      0.79       188
         1.0       0.81      0.81      0.81       212

    accuracy                           0.80       400
   macro avg       0.80      0.80      0.80       400
weighted avg       0.80      0.80      0.80       400



In [41]:
# 판별된 최적 알고리듬과 하이퍼매개변수
aml1.best_estimator, aml1.best_config

('xgb_limitdepth',
 {'colsample_bylevel': 1.0,
  'colsample_bytree': 1.0,
  'learning_rate': 0.29999999999999993,
  'max_depth': 6,
  'min_child_weight': 0.9999999999999993,
  'n_estimators': 10,
  'reg_alpha': 0.0009765625,
  'reg_lambda': 1.0,
  'subsample': 1.0})

## 6. 더 좋은 모델 찾기

In [42]:
# 더 많은 시간을 투입하여 성능이 더 좋은 모델 탐색 시도
aml2 = flaml.AutoML()
aml2.fit(X_train, y_train, task="classification", time_budget=60, verbose=False)


INFO:flaml.searcher.blendsearch:No low-cost partial config given to the search algorithm. For cost-frugal search, consider providing low-cost values for cost-related hps via 'low_cost_partial_config'. More info can be found at https://microsoft.github.io/FLAML/docs/FAQ#about-low_cost_partial_config-in-tune


In [43]:
#평가
#train data에 의해 생성된 새로운 모델을 이용하여 test data로 예측
y_pred = aml2.predict(X_test)
print(sum(abs(y_test - y_pred)))  #새로운 모델에서 
                                  #잘못 예측한 데이터 갯수 의미

74.0


In [44]:
#새로운 모델에 대해
#예측된 값과 실제값 사이의 오차 성능평가 보고서 (Accuracy 등)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.80      0.81      0.81       188
         1.0       0.83      0.82      0.82       212

    accuracy                           0.81       400
   macro avg       0.81      0.81      0.81       400
weighted avg       0.82      0.81      0.82       400



In [45]:
# 새롭게 판별된 최적 알고리듬과 하이퍼매개변수
aml2.best_estimator, aml2.best_config

('lgbm',
 {'colsample_bytree': 0.45376228256312906,
  'learning_rate': 0.037446000944520684,
  'log_max_bin': 10,
  'min_child_samples': 2,
  'n_estimators': 325,
  'num_leaves': 972,
  'reg_alpha': 0.0009765625,
  'reg_lambda': 0.0022188584653673863})

## 결론

인공 지능은 다양한 현대 사회의 문제를 해결하는데 널리 사용되고 있습니다. 분류는 인공지능에서 다루는 대표적인 지도학습 문제 유형입니다. 이 실험에서는 분류 문제의 사례인 와인 등급 분류에 인공 지능을 사용하는 방법의 
예를 보았습니다. 같은 목적으로 다른 모델을 사용할 수도 있습니다.